In [1]:
#!pip install google-cloud-vision
#!pip install opencv-python

In [2]:
import os, io
from google.cloud import vision
from google.cloud.vision_v1 import types
import cv2 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'ServiceAccountToken.json'

In [3]:
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations

    #print('Number of objects found: {}'.format(len(objects)))
    #for object_ in objects:
    #    print('\n{} (confidence: {})'.format(object_.name, object_.score))
    #    print('Normalized bounding polygon vertices: ')
    #    for vertex in object_.bounding_poly.normalized_vertices:
    #        print(' - ({}, {})'.format(vertex.x, vertex.y))
    return(objects)

In [4]:
def add_rect(image,label,x1,y1,x2,y2):
    x1 = int(x1*image.shape[1])
    y1= int(y1*image.shape[0])
    x2 = int(x2*image.shape[1])
    y2 = int(y2*image.shape[0])
    start_point = (x1,y1)
    end_point = (x2,y2)
    color = (0, 0, 255)
    thickness = 2
    image = cv2.rectangle(image, start_point, end_point, color, thickness)
    font = 0.5
    thick = 1
    if (image.shape[1] < 450):
        font = 0.3
        thick = 1
    elif (image.shape[1] < 900):
        font = 0.6
        thick = 2
    elif (image.shape[1] >= 900):
        font = 0.7
        thick = 3 
    if (y1-20 < 0):
        image = cv2.putText(image, label, (x1, y1+10), cv2.FONT_HERSHEY_SIMPLEX, font, (36,255,12), thick)
    else:
        image = cv2.putText(image, label, (x1, y1-20), cv2.FONT_HERSHEY_SIMPLEX, font, (36,255,12), thick)
    return(image)

def see(path,objects):
    image = cv2.imread(path)
    for object_ in objects:
        score = str(object_.score)
        label = object_.name +' '+ score[:4]
        count = 1
        for vertex in object_.bounding_poly.normalized_vertices:
            if count == 1:
                x1 = vertex.x
                y1 = vertex.y
            if count == 3:
                x2 = vertex.x
                y2 = vertex.y
            count += 1
        image = add_rect(image,label,x1,y1,x2,y2)
    return(image)

In [5]:
def find_prop(path,objects):
    image = see(path,objects)
    window_name = 'Image'

    scale_percent = 50000/image.shape[0]
    width = int(image.shape[1] * scale_percent / 100)
    #height = int(image.shape[0] * scale_percent / 100)
    dim = (width, 500)

    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    cv2.imshow(window_name, image) 
    cv2.waitKey(0)

In [6]:
def web_stuff(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    response = client.web_detection(image=image)
    annotations = response.web_detection
    
    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            print('\nBest guess label: {}'.format(label.label))

    if annotations.pages_with_matching_images:
        print('\n{} Pages with matching images found:'.format(
            len(annotations.pages_with_matching_images)))
        for page in annotations.pages_with_matching_images:
            if page.full_matching_images:
                for image in page.full_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))
            if page.partial_matching_images:
                for image in page.partial_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))
                    
    #if annotations.visually_similar_images:
    #    print('\n{} visually similar images found:\n'.format(
    #        len(annotations.visually_similar_images)))

    #    for image in annotations.visually_similar_images:
    #        print('\tImage url    : {}'.format(image.url))
            
    if annotations.web_entities:
        print('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print('\n\tScore      : {}'.format(entity.score))
            print(u'\tDescription: {}'.format(entity.description))

In [11]:
path = 'photos/image29.jpeg'

In [12]:
objects = localize_objects(path)
print('Number of objects found: {}'.format(len(objects)))
for object_ in objects:
    print('\n{} (confidence: {})'.format(object_.name, object_.score))
find_prop(path,objects)
web_stuff(path)

Number of objects found: 2

Top (confidence: 0.8494710326194763)

Person (confidence: 0.8207537531852722)

Best guess label: t shirt

10 Pages with matching images found:
		Image url  : https://www.mcclatchy-wires.com/incoming/b2vddh/picture265119654/alternates/LANDSCAPE_480/01gbrc05284x6j61fvnq.jpg
		Image url  : https://www.mcclatchy-wires.com/incoming/b2vddh/picture265119654/alternates/LANDSCAPE_480/01gbrc05284x6j61fvnq.jpg
		Image url  : https://i.ytimg.com/vi/UDKtbH0iTLQ/mqdefault.jpg
		Image url  : https://www.mcclatchy-wires.com/incoming/b2vddh/picture265119654/alternates/LANDSCAPE_480/01gbrc05284x6j61fvnq.jpg
		Image url  : https://www.mcclatchy-wires.com/incoming/b2vddh/picture265119654/alternates/LANDSCAPE_480/01gbrc05284x6j61fvnq.jpg
		Image url  : https://www.mcclatchy-wires.com/incoming/b2vddh/picture265119654/alternates/LANDSCAPE_480/01gbrc05284x6j61fvnq.jpg
		Image url  : https://i.ytimg.com/vi/UDKtbH0iTLQ/mqdefault.jpg
		Image url  : https://www.mcclatchy-wires.com/inco